## Svm classifier


In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import preprocessing

In [2]:
dataset = pd.read_csv('../Data/imputedWQ.csv', parse_dates=True, header=0, index_col=0)

In [3]:
testset = pd.read_csv('../Data/testing2018.csv', parse_dates=True, header=0, index_col=0)

In [4]:
X = dataset.drop('EVENT', axis=1)  
y = dataset['EVENT']  

In [5]:
# from sklearn.model_selection import train_test_split  
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30) 

In [6]:
# y_pred = svclassifier.predict(X_test)  

In [7]:
# print(classification_report(y_test,y_pred)) 

# TimeSeriesSplit
## Special case of KFold
### Param: n_splits according to the number of months

P.s: even though this probably would not result in accurate month separation, probably would go more with accurate samples separation

In [8]:
tss = TimeSeriesSplit(n_splits=4)
# kn = KNeighborsClassifier(n_neighbors=5, algorithm='brute') 
# sc = cross_val_score(kn, X, y, cv=tss, scoring='accuracy')

In [9]:
dataset.groupby(dataset.index.month).sum()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
Time,,,,,,,,,,
8,290766.552632,6801.383362,343517.849450,3.096140e+07,8.664862e+06,787160.949453,4.009652e+06,6.738802e+07,3.886867e+07,165.0
9,353961.000000,7172.786251,361367.891324,3.252038e+07,9.111793e+06,865092.986451,4.396304e+06,6.858502e+07,4.055888e+07,1310.0
10,431439.430435,7384.794348,374076.693977,3.364090e+07,9.410047e+06,710263.140783,4.398462e+06,6.299596e+07,4.047184e+07,174.0
11,112340.400000,1736.240000,88673.130000,7.957033e+06,2.056404e+06,144211.260000,9.913454e+05,1.533976e+07,9.644754e+06,77.0


In [10]:
testset.groupby(testset.index.month).sum()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
Time,,,,,,,,,,
1,168946.9,6486.070000,367918.485157,3.433636e+07,1.030073e+07,810.50142,4804.052413,65217445.0,45054296.0,773.0
2,65273.9,2560.330000,144423.580000,1.363104e+07,3.474466e+06,286.97000,1847.039119,26277783.0,18979849.0,895.0
11,183728.9,3401.380063,177387.328683,1.620504e+07,4.328124e+06,425.34816,2341.372000,30974768.0,19343072.0,352.0
12,171830.7,4713.325094,260945.894188,2.417727e+07,7.182003e+06,634.66800,3317.529000,44719245.0,26572835.0,309.0


## TimeSeries CV

- Applying TimeSeriesSplit in cross validation with multiple scores
    - F1-macro (alter. f1_weighted, f1_micro)
    - Precision
    - Recall
    
P.s errors result because the classifier on specific folds couldn't identify at all one class and probably gave just true or false ( I guess false in this case because of high class-imbalance)

In [11]:
scoring = ['f1', 'roc_auc', 'f1_macro', 'precision', 'recall']

In [12]:
scaler = preprocessing.StandardScaler()
X_train_standard = scaler.fit_transform(X)

svm_clf = SVC(C=100, gamma=0.001 )           
              
results = cross_validate(svm_clf, X, y, cv=tss, scoring=scoring, n_jobs=-1)

In [13]:
print("Model(svm, c=100, gamma=0.001): ({0}, {1}, {2}, {3},{4})\n".format(results['test_f1'].mean(), results['test_roc_auc'].mean(), results['test_f1_macro'].mean(), results['test_precision'].mean(), results['test_recall'].mean()))

Model(svm, c=100, gamma=0.001): (0.008230534536423157, 0.7975736167850658, 0.4954848766875112, 0.1829782597549095,0.0699796633359834)



In [14]:
svm_clf.fit(X_train_standard, y)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
y_pred = svm_clf.predict(scaler.transform(X))

In [16]:
f1_score(y_pred, y)
#f1_score(y_pred, y, average='macro')

0.8156462585034013

## GridSearchCV
### Hyperparameter optimizations for RandForest classifier

- Scoring F1_macro can be changed to precision, recall or f1_weighted or f1_micro and possibly you would get different parameters.

In [17]:
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

param_grid = [{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf'],'class_weight': ['balanced']},
 ]  


grid = GridSearchCV(SVC(), param_grid, cv=tss, scoring="f1_macro", n_jobs=-1)

In [18]:
grid.fit(X,y)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=4),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf'], 'class_weight': ['balanced']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_macro', verbose=0)

In [19]:
grid.best_params_

{'C': 1000, 'class_weight': 'balanced', 'gamma': 0.0001, 'kernel': 'rbf'}

In [20]:
model = grid.best_estimator_

In [21]:
results = cross_validate(model, X, y, cv=tss, scoring=scoring, n_jobs=-1)

In [22]:
print("Model(svm, c:1000,balanced, gamma=0.0001, 'rbf'): ({0}, {1}, {2})\n".format(results['test_f1'].mean(), results['test_f1_macro'].mean(), results['test_precision'].mean(), results['test_recall'].mean()))

Model(svm, c:1000,balanced, gamma=0.0001, 'rbf'): (0.1493683707425046, 0.5603742858555992, 0.33461734382891273)



# Testing on Test set

In [23]:
testset.fillna(0, inplace=True)

In [24]:
X_test = testset.drop('EVENT', axis=1)  
y_test = testset['EVENT']  

In [25]:
predicted = model.predict(X_test)

In [37]:
predicted = predicted<0.1

In [38]:
f1_score(y_test,predicted, average='macro')

0.4829012220989268

In [39]:
confusion_matrix(y_test,predicted)

array([[124660,  12577],
       [  2166,    163]])